# Ch.9 Data Aggregation and Group Operations

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [2]:
'''
When True, IPython notebook will use html representation for pandas objects 
(if it is available). [default: True] [currently: True]
'''
pd.options.display.notebook_repr_html = False

In [3]:
%matplotlib inline

## GroupBy mechanics

In [4]:
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
2 -0.519439  0.281746    b  one
3 -0.555730  0.769023    b  two
4  1.965781  1.246435    a  one

In [5]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [7]:
#group the dataframe by its value in 'key1'
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

In [8]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [9]:
means.unstack()

key2       one       two
key1                    
a     0.880536  0.478943
b    -0.519439 -0.555730

In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [11]:
df.groupby('key1').mean()

         data1     data2
key1                    
a     0.746672  0.910916
b    -0.537585  0.525384

In [12]:
df.groupby(['key1', 'key2']).mean() 
#calculate the value of each categories divided by the labels

              data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

In [13]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating over groups

In [14]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
4  1.965781  1.246435    a  one
b
      data1     data2 key1 key2
2 -0.519439  0.281746    b  one
3 -0.555730  0.769023    b  two


In [20]:
#for (k1, k2), group in df.groupby(['key1', 'key2']):
#    print((k1, k2))
#    print(group)
    
for item in df.groupby(['key1', 'key2']):
    print(item)
#    print(group)

(('a', 'one'),       data1     data2 key1 key2
0 -0.204708  1.393406    a  one
4  1.965781  1.246435    a  one)
(('a', 'two'),       data1     data2 key1 key2
1  0.478943  0.092908    a  two)
(('b', 'one'),       data1     data2 key1 key2
2 -0.519439  0.281746    b  one)
(('b', 'two'),      data1     data2 key1 key2
3 -0.55573  0.769023    b  two)


In [21]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

      data1     data2 key1 key2
2 -0.519439  0.281746    b  one
3 -0.555730  0.769023    b  two

In [22]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [24]:
grouped = df.groupby(df.dtypes, axis=1)
dict(list(grouped))

{dtype('float64'):       data1     data2
 0 -0.204708  1.393406
 1  0.478943  0.092908
 2 -0.519439  0.281746
 3 -0.555730  0.769023
 4  1.965781  1.246435, dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

### Selecting a column or subset of columns

In [25]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [26]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [29]:
df.groupby(['key1', 'key2'])[['data2']].mean()
#df.groupby(['key1', 'key2']).mean()

              data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [30]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [31]:
s_grouped.mean() # same as above.

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

### Grouping with dicts and Series

In [33]:
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.loc[2:3, ['b', 'c']] = np.nan # Add a few NA values
people

               a         b         c         d         e
Joe     0.560145 -1.265934  0.119827 -1.063512  0.332883
Steve  -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
Wes     0.286350       NaN       NaN  0.331286  1.349742
Jim     0.069877  0.246674 -0.011862  1.004812  1.327195
Travis -0.919262 -1.549106  0.022185  0.758363 -0.660524

In [34]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [36]:
by_column = people.groupby(mapping, axis=1)
#'f' is for 'orange', so there is no data for the 'orange' label
by_column.sum()

            blue       red
Joe    -0.943685 -0.372906
Steve  -2.512731 -3.865992
Wes     0.331286  1.636092
Jim     0.992950  1.643745
Travis  0.780548 -3.128892

In [41]:
map_series = Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [42]:
people

               a         b         c         d         e
Joe     0.560145 -1.265934  0.119827 -1.063512  0.332883
Steve  -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
Wes     0.286350       NaN       NaN  0.331286  1.349742
Jim     0.069877  0.246674 -0.011862  1.004812  1.327195
Travis -0.919262 -1.549106  0.022185  0.758363 -0.660524

In [38]:
people.groupby(map_series, axis=1).count() 
# return blue/red if the value exists

        blue  red
Joe        2    3
Steve      2    3
Wes        1    2
Jim        2    3
Travis     2    3

### Grouping with functions

In [43]:
people.groupby(len).sum()
# to group the data by their length of index, then sum them

          a         b         c         d         e
3  0.916372 -1.019260  0.107966  0.272585  3.009820
5 -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
6 -0.919262 -1.549106  0.022185  0.758363 -0.660524

In [44]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

              a         b         c         d         e
3 one  0.286350 -1.265934  0.119827 -1.063512  0.332883
  two  0.069877  0.246674 -0.011862  1.004812  1.327195
5 one -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
6 two -0.919262 -1.549106  0.022185  0.758363 -0.660524

### Grouping by index levels

In [47]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]], names=['cty', 'tenor'])
#Multi_Index_Method = pd.MultiIndex.from_arrays()
hier_df = DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.051316 -1.157719  0.816707  0.433610  1.010737
1      1.824875 -0.997518  0.850591 -0.131578  0.912414
2      0.188211  2.169461 -0.114928  2.003697  0.029610
3      0.795253  0.118110 -0.748532  0.584970  0.152677

In [51]:
hier_df.groupby(level='cty', axis=1).count()
# to count the numbers?

cty  JP  US
0     2   3
1     2   3
2     2   3
3     2   3

## Data aggregation

In [52]:
df

      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
2 -0.519439  0.281746    b  one
3 -0.555730  0.769023    b  two
4  1.965781  1.246435    a  one

In [53]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)
# to calculate quantile, is equal to return the (min + (max-min) * quantile)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [54]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

         data1     data2
key1                    
a     2.170488  1.300498
b     0.036292  0.487276

In [57]:
grouped.describe() # a typical statistical summary of the data

     data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023  

### Column-wise and multiple function application
This part has been clipped due to the lack of data.

### Returning aggregated data in "unindexed" form

In [60]:
#tips.groupby(['sex', 'smoker'], as_index=False).mean()

## Group-wise operations and transformations

In [63]:
'''
To add a column to a DataFrame containing group means for each index. 
One way to do this is to aggregate, then merge.
'''
df

      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
1  0.478943  0.092908    a  two
2 -0.519439  0.281746    b  one
3 -0.555730  0.769023    b  two
4  1.965781  1.246435    a  one

In [64]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

      mean_data1  mean_data2
key1                        
a       0.746672    0.910916
b      -0.537585    0.525384

In [68]:
pd.merge(df, k1_means, left_on='key1', right_index=True)
# it seems that the 'right_index=True' is a necessary sentence

      data1     data2 key1 key2  mean_data1  mean_data2
0 -0.204708  1.393406    a  one    0.746672    0.910916
1  0.478943  0.092908    a  two    0.746672    0.910916
4  1.965781  1.246435    a  one    0.746672    0.910916
2 -0.519439  0.281746    b  one   -0.537585    0.525384
3 -0.555730  0.769023    b  two   -0.537585    0.525384

In [74]:
key = ['one', 'two', 'one', 'two', 'one']
# odd and even index people
people.groupby(key).mean()

            a         b         c         d         e
one -0.024256 -1.407520  0.071006  0.008712  0.340700
two -1.144771  0.023566 -0.776929  0.017038  0.010082

In [75]:
people.groupby(key).transform(np.mean)

               a         b         c         d         e
Joe    -0.024256 -1.407520  0.071006  0.008712  0.340700
Steve  -1.144771  0.023566 -0.776929  0.017038  0.010082
Wes    -0.024256 -1.407520  0.071006  0.008712  0.340700
Jim    -1.144771  0.023566 -0.776929  0.017038  0.010082
Travis -0.024256 -1.407520  0.071006  0.008712  0.340700

In [76]:
def demean(arr):
    return arr - arr.mean()
demeaned = people.groupby(key).transform(demean)
demeaned

               a         b         c         d         e
Joe     0.584401  0.141586  0.048821 -1.072225 -0.007817
Steve  -1.214648 -0.223109 -0.765067 -0.987774 -1.317112
Wes     0.310605       NaN       NaN  0.322574  1.009042
Jim     1.214648  0.223109  0.765067  0.987774  1.317112
Travis -0.895006 -0.141586 -0.048821  0.749651 -1.001225

In [77]:
demeaned.groupby(key).mean()

       a             b             c    d             e
one  0.0  1.110223e-16  6.938894e-18  0.0  7.401487e-17
two  0.0  0.000000e+00  0.000000e+00  0.0  0.000000e+00

### Apply: General split-apply-combine

In [87]:
def top(df, n=5, column='tip_pct'):
    return df.sort_index(by=column)[-n:]

# to select the top five tip_pct(column name) values by group.

#### Suppressing the group keys

In [88]:
#tips.groupby('smoker', group_keys=False).apply(top)

### Quantile and bucket analysis

In [95]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})

factor = pd.cut(frame.data1, 4)
'''
pd.cut(x, bins, right=True, labels=None, retbins=False, 
precision=3, include_lowest=False)

bins : int, sequence of scalars, or IntervalIndex
    If `bins` is an int, it defines the number of equal-width bins in the
    range of `x`. However, in this case, the range of `x` is extended
    by .1% on each side to include the min or max values of `x`. If
    `bins` is a sequence it defines the bin edges allowing for
    non-uniform bin width. No extension of the range of `x` is done in
    this case.
    
bins = 4, which means for each data, factor will save 
        the first 4 digits of the data.
'''
factor[:10]

0    (-1.185, 0.509]
1     (0.509, 2.204]
2    (-1.185, 0.509]
3    (-1.185, 0.509]
4    (-1.185, 0.509]
5     (0.509, 2.204]
6    (-1.185, 0.509]
7     (0.509, 2.204]
8     (0.509, 2.204]
9     (0.509, 2.204]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.885, -1.185] < (-1.185, 0.509] < (0.509, 2.204] < (2.204, 3.898]]

In [91]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
# 简易版的.describe() method
grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

#ADAPT the output is not sorted in the book 
#while this is the case now (swap first two lines)

                  count       max      mean       min
data1                                                
(-3.201, -1.64]    44.0  1.562211 -0.054708 -1.614189
(-1.64, -0.0853]  401.0  3.082067 -0.033824 -3.481593
(-0.0853, 1.469]  490.0  3.189940 -0.034471 -2.939965
(1.469, 3.024]     65.0  1.398385 -0.136144 -2.246992

In [96]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

       count       max      mean       min
data1                                     
0      100.0  2.404580 -0.163764 -2.587681
1      100.0  2.040657 -0.023332 -2.904543
2      100.0  2.231810  0.142036 -2.403642
3      100.0  2.138638 -0.115840 -2.794475
4      100.0  2.267944 -0.024669 -3.255825
5      100.0  3.521136 -0.035211 -2.133437
6      100.0  2.478106 -0.022286 -2.098641
7      100.0  2.246298  0.031404 -2.924827
8      100.0  2.533112  0.154331 -3.398085
9      100.0  2.943909  0.097091 -2.938085

### Example: Filling missing values with group-specific values

In [97]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    1.844281
2         NaN
3    0.447351
4         NaN
5    0.999861
dtype: float64

In [98]:
s.fillna(s.mean())

0    1.097164
1    1.844281
2    1.097164
3    0.447351
4    1.097164
5    0.999861
dtype: float64

In [99]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4 
    #这里应该存的是4个‘East’再加上4个‘West’
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.880438
New York     -1.400894
Vermont            NaN
Florida       0.730709
Oregon        0.788423
Nevada             NaN
California    0.540007
Idaho              NaN
dtype: float64

In [100]:
data.groupby(group_key).mean()

East    0.070084
West    0.664215
dtype: float64

In [101]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.880438
New York     -1.400894
Vermont       0.070084
Florida       0.730709
Oregon        0.788423
Nevada        0.664215
California    0.540007
Idaho         0.664215
dtype: float64

In [102]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          0.880438
New York     -1.400894
Vermont       0.500000
Florida       0.730709
Oregon        0.788423
Nevada       -1.000000
California    0.540007
Idaho        -1.000000
dtype: float64

### Example: Random sampling and permutation

In [112]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q'] #Cards Value
#print(base_names)
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = Series(card_val, index=cards)

# To generate a dataframe which stores the data for a entire set of cards

In [113]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [121]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)
# here deck is a dataframe/series, so series.take
#np.random.permutation: 
#  - Randomly permute a sequence, or return a permuted range.

2C     2
JD    10
AD     1
KH    10
KS    10
dtype: int64

In [118]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  10C    10
   2C      2
D  7D      7
   6D      6
H  AH      1
   7H      7
S  QS     10
   6S      6
dtype: int64

In [119]:
# alternatively
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

KC    10
6C     6
3D     3
2D     2
4H     4
6H     6
7S     7
9S     9
dtype: int64

### Example: Group weighted average and correlation

In [ ]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})
df

In [ ]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

In [ ]:
close_px = pd.read_csv('ch09/stock_px.csv', parse_dates=True, index_col=0)
close_px.info()

In [ ]:
close_px[-4:]

In [ ]:
rets = close_px.pct_change().dropna()
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x: x.year)
by_year.apply(spx_corr)

In [ ]:
# Annual correlation of Apple with Microsoft
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### Example: Group-wise linear regression

In [ ]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [ ]:
by_year.apply(regress, 'AAPL', ['SPX'])

## Pivot tables and Cross-tabulation

In [ ]:
tips.pivot_table(index=['sex', 'smoker'])

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],
                 columns='smoker')

In [ ]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],
                 columns='smoker', margins=True)

In [ ]:
tips.pivot_table('tip_pct', index=['sex', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

In [ ]:
tips.pivot_table('size', index=['time', 'sex', 'smoker'],
                 columns='day', aggfunc='sum', fill_value=0)

### Cross-tabulations: crosstab

In [ ]:
from StringIO import StringIO
data = """\
Sample    Gender    Handedness
1    Female    Right-handed
2    Male    Left-handed
3    Female    Right-handed
4    Male    Right-handed
5    Male    Left-handed
6    Male    Right-handed
7    Female    Right-handed
8    Female    Left-handed
9    Male    Right-handed
10    Female    Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [ ]:
data

In [ ]:
pd.crosstab(data.Gender, data.Handedness, margins=True)

In [ ]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

## Example: 2012 Federal Election Commission Database

In [ ]:
fec = pd.read_csv('ch09/P00000001-ALL.csv')

In [ ]:
fec.info()

In [ ]:
fec.ix[123456]

In [ ]:
unique_cands = fec.cand_nm.unique()
unique_cands

In [ ]:
unique_cands[2]

In [ ]:
parties = {'Bachmann, Michelle': 'Republican',
           'Cain, Herman': 'Republican',
           'Gingrich, Newt': 'Republican',
           'Huntsman, Jon': 'Republican',
           'Johnson, Gary Earl': 'Republican',
           'McCotter, Thaddeus G': 'Republican',
           'Obama, Barack': 'Democrat',
           'Paul, Ron': 'Republican',
           'Pawlenty, Timothy': 'Republican',
           'Perry, Rick': 'Republican',
           "Roemer, Charles E. 'Buddy' III": 'Republican',
           'Romney, Mitt': 'Republican',
           'Santorum, Rick': 'Republican'}

In [ ]:
fec.cand_nm[123456:123461]

In [ ]:
fec.cand_nm[123456:123461].map(parties)

In [ ]:
# Add it as a column
fec['party'] = fec.cand_nm.map(parties)

In [ ]:
fec['party'].value_counts()

In [ ]:
(fec.contb_receipt_amt > 0).value_counts()

In [ ]:
fec = fec[fec.contb_receipt_amt > 0]

In [ ]:
fec_mrbo = fec[fec.cand_nm.isin(['Obama, Barack', 'Romney, Mitt'])]

### Donation statistics by occupation and employer

In [ ]:
fec.contbr_occupation.value_counts()[:10]

In [ ]:
occ_mapping = {
   'INFORMATION REQUESTED PER BEST EFFORTS' : 'NOT PROVIDED',
   'INFORMATION REQUESTED' : 'NOT PROVIDED',
   'INFORMATION REQUESTED (BEST EFFORTS)' : 'NOT PROVIDED',
   'C.E.O.': 'CEO'
}

# If no mapping provided, return x
f = lambda x: occ_mapping.get(x, x)
fec.contbr_occupation = fec.contbr_occupation.map(f)

In [ ]:
emp_mapping = {
   'INFORMATION REQUESTED PER BEST EFFORTS' : 'NOT PROVIDED',
   'INFORMATION REQUESTED' : 'NOT PROVIDED',
   'SELF' : 'SELF-EMPLOYED',
   'SELF EMPLOYED' : 'SELF-EMPLOYED',
}

# If no mapping provided, return x
f = lambda x: emp_mapping.get(x, x)
fec.contbr_employer = fec.contbr_employer.map(f)

In [ ]:
by_occupation = fec.pivot_table('contb_receipt_amt',
                                index='contbr_occupation',
                                columns='party', aggfunc='sum')

In [ ]:
over_2mm = by_occupation[by_occupation.sum(1) > 2000000]
over_2mm

In [ ]:
over_2mm.plot(kind='barh')

In [ ]:
def get_top_amounts(group, key, n=5):
    totals = group.groupby(key)['contb_receipt_amt'].sum()

    # Order totals by key in descending order
    return totals.order(ascending=False)[-n:]

In [ ]:
grouped = fec_mrbo.groupby('cand_nm')
grouped.apply(get_top_amounts, 'contbr_occupation', n=7)

In [ ]:
grouped.apply(get_top_amounts, 'contbr_employer', n=10)

### Bucketing donation amounts

In [ ]:
bins = np.array([0, 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000])
labels = pd.cut(fec_mrbo.contb_receipt_amt, bins)
labels

In [ ]:
grouped = fec_mrbo.groupby(['cand_nm', labels])
grouped.size().unstack(0)

In [ ]:
bucket_sums = grouped.contb_receipt_amt.sum().unstack(0)
bucket_sums

In [ ]:
normed_sums = bucket_sums.div(bucket_sums.sum(axis=1), axis=0)
normed_sums

In [ ]:
normed_sums[:-2].plot(kind='barh', stacked=True)

### Donation statistics by state

In [ ]:
grouped = fec_mrbo.groupby(['cand_nm', 'contbr_st'])
totals = grouped.contb_receipt_amt.sum().unstack(0).fillna(0)
totals = totals[totals.sum(1) > 100000]
totals[:10]

In [ ]:
percent = totals.div(totals.sum(1), axis=0)
percent[:10]